In [1]:
import argparse
import os
import sys
from os import mkdir
from apex import amp
import shutil



import torch.nn.functional as F

sys.path.append('..')
from config import cfg
from data import make_data_loader, make_data_loader_view
from engine.trainer import do_train
from modeling import build_model
from solver import make_optimizer, WarmupMultiStepLR,build_scheduler
from layers import make_loss

from utils.logger import setup_logger


from torch.utils.tensorboard import SummaryWriter
import torch
from layers.RaySamplePoint import RaySamplePoint
from engine import render
import numpy as np 
torch.cuda.set_device(2)
import matplotlib.pyplot as plt
import cv2
from tqdm.notebook import trange, tqdm
from imageio_ffmpeg import write_frames
import trimesh

In [2]:
def vis_density(model, maxs, mins, L= 32):
    
#     maxs = (np.array([1.0, 1.0, 1.0]) - np.array([0.4938117, 0.5103138,0.39479887]))/np.array([0.3771917,0.3771917,0.3771917])
#     mins = (np.array([0.0, 0.0, 0.0]) - np.array([0.4938117, 0.5103138,0.39479887]))/np.array([0.3771917,0.3771917,0.3771917])
    
    x = torch.linspace(mins[0],maxs[0],steps=L).cuda()
    y = torch.linspace(mins[1],maxs[1],steps=L).cuda()
    z = torch.linspace(mins[2],maxs[2],steps=L).cuda()
    grid_x ,grid_y,grid_z = torch.meshgrid(x, y,z)
    xyz = torch.stack([grid_x ,grid_y,grid_z], dim = -1)  #(L,L,L,3)

    xyz = xyz.reshape((-1,3)) #(L*L*L,3)

    xyzs = xyz.split(5000, dim=0)

    sigmas = []
    coeffs = []
    for i in xyzs:
        with torch.no_grad():
            _,density, coeff = model.spacenet_fine(i, None) #(L*L*L,1)
            density = torch.nn.functional.relu(density)
            sigmas.append(density.detach().cpu())
            coeffs.append(coeff.detach().cpu())
            
    sigmas = torch.cat(sigmas, dim=0)
    coeffs = torch.cat(coeffs, dim=0)

    return sigmas, coeffs

In [3]:
training_folder = '/data/new_disk/suoxin/output/lego_sh5'
epoch = 208000
para_file = 'rfnr_checkpoint_%d.pt' % epoch

In [4]:
cfg.merge_from_file(os.path.join(training_folder,'configs.yml'))
cfg.freeze()

In [5]:
val_loader, dataset_val = make_data_loader_view(cfg, is_train=False)
train_loader, dataset = make_data_loader(cfg, is_train=True)

0 / 1
load 100 Ts, 100 Ks, 1 frame, 421230 vertices
dataset initialed. near: 2.000000  far: 6.000000
0 / 1
load 100 Ts, 100 Ks, 1 frame, 421230 vertices
dataset initialed. near: 2.000000  far: 6.000000
load 16000000 rays.


In [6]:
model = build_model(cfg).cuda()
model.load_state_dict(torch.load(os.path.join(training_folder,para_file),map_location='cpu')['model'])
model.eval()
model.cuda()
pass

In [7]:
maxs = (np.array([1.0, 1.0, 1.0]) - np.array([0.4938117, 0.5103138,0.39479887]))/np.array([0.3771917,0.3771917,0.3771917])
mins = (np.array([0.0, 0.0, 0.0]) - np.array([0.4938117, 0.5103138,0.39479887]))/np.array([0.3771917,0.3771917,0.3771917])

In [8]:
L = 256
sigma, coeffs = vis_density(model, maxs, mins, L)

In [9]:
class TreeNode:
    def __init__(self, flag=0, xyz = (0,0,0)):
        self.node_list = None
        self.xyz = xyz
        self.flag = flag

global grid

def gen_octree(x, y, z, size):
    
    
    if(size<=2):
        node_list = []
        flag=False
        for i in range(x, x+2):
            for j in range(y, y+2):
                for k in range(z, z+2):
                    node_list.append(TreeNode(flag=0, xyz = (i,j,k)))
                    if(grid[i,j,k]):
                        flag = True
        
        node = TreeNode(1, (x+size//2, y+size//2, z+size//2))
        if(flag==True):
            node.node_list = node_list
        else:
            node.node_list = None
            node.flag=0
        return flag, node

    flag = False
    node_list = []
    
    cnt=1
    for i in range(x, x+size, size//2):
        for j in range(y, y+size, size//2):
            for k in range(z, z+size, size//2):
                _flag, node = gen_octree(i, j, k, size//2)
                
                node_list.append(node)
                if(_flag==True):
                    flag = _flag
                cnt+=1
    
    node = TreeNode(1, (x+size//2, y+size//2, z+size//2))
    if(flag):
        node.node_list = node_list
    else:
        node.flag=0
    
    return flag, node

grid = sigma.reshape((L, L, L))>0.1
flag, node = gen_octree(0,0,0,L)


In [10]:
def get_depth(node, max_cnt, cur_cnt):
    if(node.flag==0):
        max_cnt = max(max_cnt, cur_cnt)
        return max_cnt
    else:
        cnt_tmp =0
        for i in range(len(node.node_list)):
            cnt_tmp = max(cnt_tmp, get_depth(node.node_list[i], max_cnt, cur_cnt+1))
        
        max_cnt = max(max_cnt, cnt_tmp)
        
        return max_cnt
    
print(get_depth(node, 0, 0))    
    

8


In [11]:
import queue
def BFS(node, coeffs):
    q = queue.Queue()
    cnt_q = queue.Queue()
    q.put(node)
    cnt_q.put(0)
    cnt=1
    total_cnt=0
    child = []
    density_coeff = []
    while(not q.empty()):
        cnt_tmp = cnt
        cnt = 0
        for i in range(cnt_tmp):
            node = q.get()
            cnt_num = cnt_q.get()
            
            for j in range(len(node.node_list)):
                xyz = node.node_list[j].xyz
                
                density_coeff.append(coeffs[xyz[0], xyz[1], xyz[2]])
                if(node.node_list[j].flag==1):
                    total_cnt+=1
                    q.put(node.node_list[j])
                    cnt_q.put(total_cnt)
                    child.append(total_cnt - cnt_num)
                    cnt += 1
                else:
                    child.append(0)
                    
    child = np.array(child).reshape((-1,2,2,2))
    density_coeff = np.array(density_coeff)
    density_coeff = density_coeff.reshape((-1,2,2,2,28))
    return child, density_coeff

In [12]:
coeffs = coeffs.reshape((L, L, L, 28)).cpu().numpy()

In [15]:

child, density_coeff = BFS(node, coeffs)    
data = {}
data['invradius3'] = np.array([0.3771917, 0.3771917, 0.3771917]).astype(np.float32)
data['offset'] = np.array([0.4938117,  0.5103138,  0.39479887]).astype(np.float32)
data['data_format'] = "SH9"
data['data_dim'] = 28
data['child'] = child.astype(np.float32)


density_coeff1 = density_coeff[:,:,:,:, 1:28].reshape((-1, 2,2,2,9, 3))
density_coeff1 = np.concatenate((density_coeff1[:,:,:,:,:,0],density_coeff1[:,:,:,:,:,1],density_coeff1[:,:,:,:,:,2]), axis=4)
data['data'] = np.concatenate((density_coeff1, density_coeff[:,:,:,:, 0:1].reshape((density_coeff.shape[0], density_coeff.shape[1], density_coeff.shape[2], density_coeff.shape[3], 1))), axis=4)

np.savez('./lego3.npz', invradius3 = data['invradius3'], offset = data['offset'], data_format = data['data_format'], data_dim = data['data_dim'], child = data['child'].astype(np.int32), data = data['data'].astype(np.float16))   